# 2-2,自动微分机制

神经网络通常依赖反向传播求梯度来更新网络参数，求梯度过程通常是一件非常复杂而容易出错的事情。

而深度学习框架可以帮助我们自动地完成这种求梯度运算。

Pytorch一般通过反向传播 backward 方法 实现这种求梯度计算。该方法求得的梯度将存在对应自变量张量的grad属性下。

除此之外，也能够调用torch.autograd.grad 函数来实现求梯度计算。

这就是Pytorch的自动微分机制。

In [1]:
import torch 
print("torch.__version__="+torch.__version__) 

torch.__version__=2.0.0+cu117


### 一，利用backward方法求导数

backward 方法通常在一个标量张量上调用，该方法求得的梯度将存在对应自变量张量的grad属性下。

如果调用的张量非标量，则要传入一个和它同形状 的gradient参数张量。

相当于用该gradient参数张量与调用张量作向量点乘，得到的标量结果再反向传播。


**1, 标量的反向传播**

In [2]:
import numpy as np 
import torch 

# f(x) = a*x**2 + b*x + c的导数

x = torch.tensor(0.0,requires_grad = True) # x需要被求导
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
y = a*torch.pow(x,2) + b*x + c 

y.backward()
dy_dx = x.grad
print(dy_dx)


tensor(-2.)


**2, 非标量的反向传播**

In [10]:
import numpy as np 
import torch 

# f(x) = a*x**2 + b*x + c

x = torch.tensor([[0.0,0.0],[1.0,2.0]],requires_grad = True) # x需要被求导
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
y = a*torch.pow(x,2) + b*x + c 

gradient = torch.tensor([[1.0,1.0],[1.0,1.0]])

print("x:\n",x)
print("y:\n",y)
y.backward(gradient = gradient)
x_grad = x.grad
print("x_grad:\n",x_grad)

x:
 tensor([[0., 0.],
        [1., 2.]], requires_grad=True)
y:
 tensor([[1., 1.],
        [0., 1.]], grad_fn=<AddBackward0>)
x_grad:
 tensor([[-2., -2.],
        [ 0.,  2.]])


In [2]:
import torch
from torch.autograd import Variable as v
x = torch.tensor(torch.ones(2, 2), requires_grad=True)
y = x + 1
y.backward()

/tmp/ipykernel_11468/2475385514.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(torch.ones(2, 2), requires_grad=True)


RuntimeError: grad can be implicitly created only for scalar outputs

In [9]:
import torch
m = torch.tensor([[2.,3.]],requires_grad=True)
print(m.tolist())
n = torch.zeros(1, 2)
n[0, 0] = m[0, 0] ** 2
n[0, 1] = m[0, 1] ** 3
n.backward(torch.tensor([[1.,1.]]))
print(m.grad)

[[2.0, 3.0]]
tensor([[ 4., 27.]])


In [7]:
import torch
m = torch.tensor([[2., 3.]], requires_grad=True)
j = torch.zeros(2 ,2)
k = torch.zeros(1, 2)

k[0, 0] = m[0, 0] ** 2 + 3 * m[0 ,1]
k[0, 1] = m[0, 1] ** 2 + 2 * m[0, 0]
k.backward(torch.tensor([[1., 1.]]))

m.grad

RuntimeError: grad can be implicitly created only for scalar outputs

**3, 非标量的反向传播可以用标量的反向传播实现**

In [12]:
import numpy as np 
import torch 

# f(x) = a*x**2 + b*x + c

x = torch.tensor([[0.0,0.0],[1.0,2.0]],requires_grad = True) # x需要被求导
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
y = a*torch.pow(x,2) + b*x + c 

gradient = torch.tensor([[1.0,1.0],[1.0,1.0]])
z = torch.sum(y*gradient)

print("x:",x)
print("y:",y)
print("z:",z)
z.backward()
x_grad = x.grad
print("x_grad:\n",x_grad)


x: tensor([[0., 0.],
        [1., 2.]], requires_grad=True)
y: tensor([[1., 1.],
        [0., 1.]], grad_fn=<AddBackward0>)
z: tensor(3., grad_fn=<SumBackward0>)
x_grad:
 tensor([[-2., -2.],
        [ 0.,  2.]])


### 二，利用autograd.grad方法求导数

In [19]:
import numpy as np 
import torch 

# f(x) = a*x**2 + b*x + c的导数

x = torch.tensor(0.0,requires_grad = True) # x需要被求导
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
y = a*torch.pow(x,2) + b*x + c


# create_graph 设置为 True 将允许创建更高阶的导数 
dy_dx = torch.autograd.grad(y,x,create_graph=True)
print(dy_dx)

# 求二阶导数
dy2_dx2 = torch.autograd.grad(dy_dx,x)

print(dy2_dx2)



(tensor(-2., grad_fn=<AddBackward0>),)
(tensor(2.),)


In [21]:
import numpy as np 
import torch 

x1 = torch.tensor(1.0,requires_grad = True) # x需要被求导
x2 = torch.tensor(2.0,requires_grad = True)

y1 = x1*x2
y2 = x1+x2


# 允许同时对多个自变量求导数
(dy1_dx1,dy1_dx2) = torch.autograd.grad(outputs=y1,inputs = [x1,x2],retain_graph = True)
print(dy1_dx1,dy1_dx2)

(dy2_dx1,dy2_dx2) = torch.autograd.grad(outputs=y2,inputs = [x1,x2],retain_graph = True)
print(dy2_dx1,dy2_dx2)

# 如果有多个因变量，相当于把多个因变量的梯度结果求和
(dy12_dx1,dy12_dx2) = torch.autograd.grad(outputs=[y1,y2],inputs = [x1,x2])
print(dy12_dx1,dy12_dx2)



tensor(2.) tensor(1.)
tensor(1.) tensor(1.)
tensor(3.) tensor(2.)


### 三，利用自动微分和优化器求最小值

In [41]:
import numpy as np 
import torch
import tqdm
# f(x) = a*x**2 + b*x + c的最小值

x = torch.tensor(0.0,requires_grad = True) # x需要被求导
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
expect_value = torch.tensor(500.0)

optimizer = torch.optim.SGD(params=[x],lr = 0.0001)


def f(x):
    result = a*torch.pow(x,2) + b*x + c 
    return(result)

for _ in tqdm.tqdm(range(100000)):
    optimizer.zero_grad()
    y = (f(x) - expect_value) ** 2
    y.backward()
    optimizer.step()
    # print(x, y, a, b, c)
   
    
print("y=",f(x).data,";","x=",x.data)


100%|██████████| 100000/100000 [00:05<00:00, 19595.59it/s]

y= tensor(499.9999) ; x= tensor(-21.3607)


**如果本书对你有所帮助，想鼓励一下作者，记得给本项目加一颗星星star⭐️，并分享给你的朋友们喔😊!** 

如果对本书内容理解上有需要进一步和作者交流的地方，欢迎在公众号"算法美食屋"下留言。作者时间和精力有限，会酌情予以回复。

也可以在公众号后台回复关键字：**加群**，加入读者交流群和大家讨论。

![算法美食屋logo.png](https://tva1.sinaimg.cn/large/e6c9d24egy1h41m2zugguj20k00b9q46.jpg)